In [1]:
with open("Day09.txt") as file:
    data = list(map(int, file.read()))

In [2]:
memory, file = [], 0
for index, blocks in enumerate(data):
    if index % 2:
        memory.extend([None] * blocks)
    else:
        memory.extend([file] * blocks)
        file += 1

In [3]:
def defrag(memory):
    memory = memory[::]
    cursor, end = 0, len(memory) - 1
    for i, file in enumerate(memory[::-1]):
        if file is None:
            continue
        file_index = end - i
        for j, block in enumerate(memory[cursor:]):
            block_index = cursor + j
            if block_index >= file_index:
                return memory
            if block is not None:
                continue
            memory[block_index] = memory[file_index]
            memory[file_index] = None
            cursor += j
            break
    return memory

def checksum(memory):
    return sum(index * file for index, file in enumerate(memory) if file is not None)

In [4]:
%%time
checksum(defrag(memory))

CPU times: user 3.79 s, sys: 0 ns, total: 3.79 s
Wall time: 3.8 s


6356833654075

In [5]:
from collections import defaultdict, Counter

def defrag_block(memory):
    memory = memory[::]
    file_blocks, empty_blocks, empty_index = defaultdict(list), Counter(), 0
    for index, block in enumerate(memory):
        if block is None:
            empty_index = empty_index or index
            empty_blocks[empty_index] += 1
        else:
            file_blocks[block].append(index)
            empty_index = None
    cursor, end = 0, len(memory) - 1
    for file, blocks in sorted(file_blocks.items(), reverse=True):
        file_size, file_start = len(blocks), blocks[0]
        for index, empty_size in sorted(empty_blocks.items()):
            if empty_size < file_size:
                continue
            if index >= file_start:
                break
            memory[index:index+file_size] = [file] * file_size
            memory[file_start:file_start+file_size] = [None] * file_size
            empty_blocks.pop(index)
            if empty_size > file_size:
                empty_blocks[index+file_size] = empty_size - file_size
            break
    return memory

In [6]:
%%time
checksum(defrag_block(memory))

CPU times: user 3.42 s, sys: 1 ms, total: 3.42 s
Wall time: 3.43 s


6389911791746